In [ ]:
import gdown 
import pandas as pd
from sklearn import metrics

from google.cloud import storage
def download_public_file(bucket_name, source_blob_name, destination_file_name):
    storage_client = storage.Client.create_anonymous_client()

    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)

    print(
        "Downloaded public blob {} from bucket {} to {}.".format(
            source_blob_name, bucket.name, destination_file_name
        )
    )

download_public_file('inspirit-ai-data-bucket-1','Data/AI Scholars/Sessions 1 - 5/Session 2b - Logistic Regression/cancer.csv','cancer_data.csv')

data = pd.read_csv('cancer_data.csv')
data['diagnosis'].replace({'M':1, 'B':0}, inplace = True)
data.to_csv('cancer_data.csv')
del data

In [ ]:

import os             
import numpy as np    
import pandas as pd   
from sklearn.metrics import accuracy_score   


In [ ]:
data_path  = 'cancer_data.csv'

dataframe = pd.read_csv(data_path)

dataframe = dataframe[['diagnosis', 'perimeter_mean', 'radius_mean', 'texture_mean', 'area_mean', 'smoothness_mean', 'concavity_mean', 'symmetry_mean']]
dataframe['diagnosis_cat'] = dataframe['diagnosis'].astype('category').map({1: '1 (malignant)', 0: '0 (benign)'})

In [ ]:
dataframe.head()

In [ ]:
dataframe.info()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt 

In [ ]:
sns.catplot(x = 'radius_mean', y = 'diagnosis_cat', data = dataframe, order=['1 (malignant)', '0 (benign)'])
dataframe.head()

In [ ]:
from sklearn import linear_model

X,y = dataframe[['radius_mean']], dataframe[['diagnosis']]

model = linear_model.LinearRegression()
model.fit(X, y)
preds = model.predict(X)

sns.scatterplot(x='radius_mean', y='diagnosis', data=dataframe)
plt.plot(X, preds, color='r')
plt.legend(['Linear Regression Fit', 'Data'])

In [ ]:
target_boundary = 15 


sns.catplot(x = 'radius_mean', y = 'diagnosis_cat', data = dataframe, order=['1 (malignant)', '0 (benign)'])
plt.plot([target_boundary, target_boundary], [-.2, 1.2], 'g', linewidth = 2) 

In [ ]:
def boundary_classifier(target_boundary, radius_mean_series):
  predictions = []
  for radius_mean in radius_mean_series:
    if radius_mean > target_boundary:
      predictions.append(1)
    else:
      predictions.append(0)
  return predictions

In [ ]:
chosen_boundary = 11.5 

y_pred = boundary_classifier(chosen_boundary, dataframe['radius_mean'])
dataframe['predicted'] = y_pred

y_true = dataframe['diagnosis']

sns.catplot(x = 'radius_mean', y = 'diagnosis_cat', hue = 'predicted', data = dataframe, order=['1 (malignant)', '0 (benign)'])
plt.plot([chosen_boundary, chosen_boundary], [-.2, 1.2], 'g', linewidth = 2)

In [ ]:
print (list(y_true))
print (y_pred)

In [ ]:
accuracy = accuracy_score(y_true,y_pred)
print(accuracy)

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(dataframe, test_size = 0.2, random_state = 1)

In [ ]:
print('Number of rows in training dataframe:', train_df.shape[0])
train_df.head()

In [ ]:
print('Number of rows in test dataframe:', test_df.shape[0])
test_df.head()

In [ ]:
X = ['radius_mean']
y = 'diagnosis'

X_train = train_df[X]
print('X_train, our input variables:')
print(X_train.head())
print()

y_train = train_df[y]
print('y_train, our output variable:')
print(y_train.head())

In [ ]:
logreg_model = linear_model.LogisticRegression()

In [ ]:
logreg_model = linear_model.LogisticRegression()
logreg_model.fit(X_train, y_train)

In [ ]:
X_test = test_df[X]
y_test = test_df[y]

In [ ]:
y_pred = logreg_model.predict(X_test)

In [ ]:
test_df['predicted'] = y_pred.squeeze()
sns.catplot(x = X[0], y = 'diagnosis_cat', hue = 'predicted', data=test_df, order=['1 (malignant)', '0 (benign)'])

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

In [ ]:
y_prob = logreg_model.predict_proba(X_test)
X_test_view = X_test[X].values.squeeze()
plt.xlabel('radius_mean')
plt.ylabel('Predicted Probability')
sns.scatterplot(x = X_test_view, y = y_prob[:,1], hue = y_test, palette=['purple','green'])

In [ ]:
dataframe.head(1)

In [ ]:
X = ['radius_mean']
y = 'diagnosis'

X_train, X_test = train_df[X], test_df[X]
y_train, y_test = train_df[y], test_df[y]

model = linear_model.LogisticRegression()
model.fit(X_train, y_train)
preds = model.predict(X_test)

accuracy = accuracy_score(y_test, preds)
print(X[0], accuracy)

In [ ]:
dataframe.head(1)

In [ ]:
train_df, test_df = train_test_split(dataframe, test_size = 0.2, random_state = 1)
X = ['perimeter_mean', 'radius_mean', 'texture_mean','area_mean']
y = 'diagnosis'

X_train, X_test = train_df[X], test_df[X]
y_train, y_test = train_df[y], test_df[y]

model = linear_model.LogisticRegression()
model.fit(X_train, y_train)
preds = model.predict(X_test)

accuracy = accuracy_score(y_test, preds)
print(X)
print (accuracy)

In [ ]:
from sklearn import metrics

cnf_matrix = metrics.confusion_matrix(y_test, y_pred)

class_names = [0,1] 

fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names)) 
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g') # Creating heatmap
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y = 1.1)
plt.ylabel('Actual diagnosis')
plt.xlabel('Predicted diagnosis')

In [ ]:
print(cnf_matrix)
(tn, fp), (fn, tp) = cnf_matrix
print ("TN, FP, FN, TP:", tn, fp, fn, tp)

In [ ]:
accuracy = (tp + tn)/(tn + fp + fn + tp)
precision = (tp)/(tp + fp)
recall = tp/(tp + fn)

print ("accuracy, precision, recall", accuracy, precision, recall)

In [ ]:
from sklearn import tree

class_dt = tree.DecisionTreeClassifier(max_depth=3)

class_dt.fit(X_train, y_train)

In [ ]:
plt.figure(figsize=(13,8))
tree.plot_tree(class_dt, fontsize=10)

In [ ]:
y_pred = class_dt.predict(X_test)

In [ ]:
print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
print("Precision: ", metrics.precision_score(y_test, y_pred))
print("Recall: ", metrics.recall_score(y_test, y_pred))